In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import deepmatcher as dm
import pandas as pd
from utils.explainer import Explainer

In [24]:
attributes =['title','authors','venue','year']

## Rnn model
We choose rnn model because it had strange rankings

In [19]:
rnn_model = dm.MatchingModel(attr_summarizer='hybrid')
rnn_model.load_state('../../models/dblp_acm_hybrid.pth')

In [28]:
testpos_nn_rnn = pd.read_csv('experiment_results/testposNNHybrid_filtered.csv')
negatives = pd.read_csv('../../Structured/DBLP-ACM/negatives.csv')

In [21]:
explainer = Explainer(rnn_model,5)

In [29]:
%%capture
critical_pos_rnn = {}
for att in attributes:
    critical_values = explainer.generateExplanations(testpos_nn_rnn,negatives,att,
                                           0.03,'../../Structured/DBLP-ACM/test_positives.csv',1)
    critical_pos_rnn[att] = critical_values

In [30]:
testneg_nn_rnn = pd.read_csv('experiment_results/testneg_nn_rnn.csv')
positives = pd.read_csv('../../Structured/DBLP-ACM/positives.csv')

In [35]:
%%capture
critical_neg_rnn = {}
for att in attributes:
    critical_values = explainer.generateExplanations(testneg_nn_rnn,positives,att,0.03,
                                           '../../Structured/DBLP-ACM/test_negatives.csv',0)
    critical_neg_rnn[att] = critical_values

In [50]:
for df,lval,rval in critical_neg_rnn['venue']:
    print(lval+'-'+rval)
    print(df.shape)

sigmod conference-international conference on management of data
(24, 10)
sigmod conference-international conference on management of data
(24, 10)
vldb j.-the vldb journal -- the international journal on very large data bases
(32, 10)
vldb j.-the vldb journal -- the international journal on very large data bases
(32, 10)
vldb-very large data bases
(21, 10)
